In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
### set working directory and add to path
import sys
import os
import time
from tqdm import tqdm

path="/content/drive/MyDrive/cod1/RL_DS/pyworld2-main"
os.chdir(path)
sys.path.append(path+"/")

In [4]:
from world2_switch import run_full, run_analisis
from pyworld2 import world2

In [5]:
#P=[BRN, NRUN, FC, CIGN,POLN]

P=[0.04, 0.99, 0.99, 0.05, 0.99]  ##default scenario  ### el segundo se sale del limite
print(run_full(output_file=None))
print(run_full(P))


0.7969945682062864
0.7996709941058557


In [6]:
# prompt: how is the fuill name skopt to install

!pip install --quiet scikit-optimize


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.4 MB/s eta 0:00:00


In [7]:

import numpy as np
from skopt import gp_minimize



# Define the objective function
def objective(params):
    P = list(params)
    #print("P", P) # Debug: print parameters
    try:
        res = run_full(P)*-1  # Assuming run_full accepts P as a parameter
        #print(f"res: {res}") #Debug
        # Replace with your actual objective function
        # This example minimizes the 'pop' value
        #objective = res['pop'][-1]
        objective = res
        return objective
    except Exception as e:
        print(f"An error occurred during optimization: {e}")
        # Handle the error appropriately, e.g., return a large value
        return 1e10  # Return a very large value to penalize failures


# Define the bounds for the parameters
Lim = [[0.02,0.04],[0.1,1.0],[0.6,1.25],[0.02,0.04],[0.1,1.0]]
#P=[BRN, NRUN, FC, CIGN, POLN]
## default:
### escenario base se sale CIGN o es CIDN
### escenario 4 se sale NRUN
### escenario 6 se sale CIGN

In [8]:
# import time

# t=time.time()


# # Bayesian Optimization
# print("Starting Bayesian Optimization")
# res_gp = gp_minimize(objective_function, Lim, n_calls=10, random_state=0
#                      )
# #print("Bayesian Optimization Result:", res_gp)
# best_params_gp = res_gp.x
# print("Best parameters (Bayesian):", best_params_gp)
# #Run analysis with the best params
# print(run_full(best_params_gp))

# t2=time.time()
# print(t2-t)


In [9]:
# from scipy.optimize import differential_evolution
# # # # Differential Evolution
# # print("Starting Differential Evolution")
# result = differential_evolution(objective, bounds=Lim, maxiter = 100, popsize=10)
# #print("Differential Evolution Result:", result)
# print("best_params_de:",result.x)
# print(run_full(  result.x))
# result.nit
# # print("Best Parameters (DE):", best_params_de)

In [10]:
# P_DE=[0.02, 0.1, 0.6, 0.04, 1.0]
# P_BO=[0.02, 0.1, 0.6, 0.04, 1.0]
# P_carlos=[0.020177589389082172, 0.10177168424599459, 1.2430886669649146, 0.02017235550729662, 0.9260494618856037]


# print(f'Algo: Differential Evolution, tiempo de ejecución una corrida 600 seg: {run_full(P_DE)}')
# print(f'Algo: Bayesian Optimization, tiempo de ejecución una corrida 104 seg: {run_full(P_BO)}')
# print(run_full(P_carlos)) ## el que había encontrado CArlos Homologando parámetros a modificar

In [14]:
import pandas as pd
import numpy as np
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt



# --- 2. Set up the Experiment ---
N_RUNS = 3
MAX_ITER = 40
all_runs_data = [] # A list to store data from all runs and iterations.

print(f"Starting {N_RUNS} runs of Differential Evolution...")

# --- 3. Run the Experiment ---
for i in tqdm(range(N_RUNS)):

    time1=time.time()

    time_start = [0] # A list to hold the start time of each run.
    time_end = [0]
    time_start[0] = time.time()

    # CORRECTED PART: Use a list to hold the counter
    counter = [0]

    def callback_fn(xk, convergence):

        objective_value = objective(xk)
        # Access and modify the counter via its index.


        time_end[0] = time.time()
        time_it = np.round(time_end[0] - time_start[0],3)
        time_start[0]=time_end[0]

        all_runs_data.append([i, counter[0], objective_value, time_it])

        counter[0] += 1

    # Run the differential evolution algorithm
    result = differential_evolution(
        objective,
        bounds=Lim,
        maxiter=MAX_ITER,
        callback=callback_fn,
        disp=False,
        tol=0
    )
    time2 = time.time()
    time_it2 = np.round(time2 - time1,3)
    print(f"Finished Run {i+1}/{N_RUNS}. Final value: {result.fun:.4f} Tiem Run: {time_it2:2f}")



# --- 4. Create and Analyze the DataFrame ---
df_results = pd.DataFrame(all_runs_data, columns=['Run', 'Iteration', 'Objective Value', "tiempo_it_seg"])



grouped_by_iter = df_results.groupby('Iteration')['Objective Value']
median_values = grouped_by_iter.quantile(0.5)*-1
lower_quantile = grouped_by_iter.quantile(0.05)*-1
upper_quantile = grouped_by_iter.quantile(0.95)*-1

Starting 3 runs of Differential Evolution...


 33%|███▎      | 1/3 [07:40<15:21, 460.50s/it]

Finished Run 1/3. Final value: -1.4340 Tiem Run: 460.501000


 67%|██████▋   | 2/3 [15:24<07:42, 462.79s/it]

Finished Run 2/3. Final value: -1.4340 Tiem Run: 464.394000


100%|██████████| 3/3 [23:07<00:00, 462.45s/it]

Finished Run 3/3. Final value: -1.4340 Tiem Run: 462.444000



CPU: 94.211000

v6e-1 TPU: 38.387000

A100 GPU: 68.901000



In [ ]:
df_results.to_csv('resultados/DE_world2_results.csv', index=False)
df_results.to_excel('resultados/DE_world2_results.xlsx', index=False)

In [ ]:

# --- 5. Graph the Results ---
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(12, 7))

ax.plot(median_values.index, median_values, 'o-', color='blue', label='Median Objective Value')
ax.fill_between(
    lower_quantile.index,
    lower_quantile,
    upper_quantile,
    color='blue',
    alpha=0.2,
    label='90% Confidence Interval (5th-95th percentile)'
)

ax.set_title(f'Differential Evolution Performance across {N_RUNS} Runs', fontsize=16)
ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Objective Value', fontsize=12)
ax.legend(fontsize=10)
#ax.set_xticks(range(MAX_ITER + 1))
#plt.yscale('log')
plt.tight_layout()
plt.show()

In [17]:
!lscpu | grep "Model name"
!free -h | grep "Mem:"
!python --version
!cat /etc/os-release | grep "PRETTY_NAME"


Model name:                           AMD EPYC 9B14
Mem:           172Gi       1.8Gi       159Gi       5.0Mi        11Gi       169Gi
Python 3.12.11
PRETTY_NAME="Ubuntu 22.04.5 LTS"
